# Autenticação e Segurança

**Referência:** Seção 00 - Fundamentos

## Objetivos de Aprendizagem

Ao final deste tutorial, você será capaz de:
- Obter e configurar uma API key da OpenAI
- Configurar variáveis de ambiente de forma segura
- Implementar boas práticas de segurança
- Gerenciar custos e limites

## Pré-requisitos

- Conta na OpenAI (https://platform.openai.com/)
- Python 3.7+ instalado
- Biblioteca `openai` e `python-dotenv` instaladas


## 2. Configuração Segura com Variáveis de Ambiente

### Por que usar variáveis de ambiente?

- **Segurança**: Evita expor chaves no código
- **Flexibilidade**: Diferentes chaves para dev/prod
- **Boas Práticas**: Padrão da indústria
- **Git Safety**: Não precisa estar no .gitignore se nunca foi commitado



### Exemplo 1: Carregar variáveis de ambiente

Usando `python-dotenv` para carregar o arquivo `.env`:


In [ ]:
import os
from dotenv import load_dotenv

# Carregar variáveis do arquivo .env
load_dotenv()

# Obter API key
api_key = os.getenv('OPENAI_API_KEY')

if api_key:
    print("✅ API key carregada com sucesso!")
    print(f"Chave: {api_key[:7]}...{api_key[-4:]}")
else:
    print("❌ API key não encontrada!")
    print("Verifique se o arquivo .env existe e contém OPENAI_API_KEY")


### Exemplo 2: Inicializar cliente OpenAI

Usar a API key carregada para inicializar o cliente:


In [ ]:
from openai import OpenAI

# Inicializar cliente com API key do ambiente
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

print("✅ Cliente OpenAI inicializado!")

# Testar conexão (opcional)
try:
    response = client.models.list()
    print(f"✅ Conexão bem-sucedida! {len(response.data)} modelos disponíveis.")
except Exception as e:
    print(f"❌ Erro ao conectar: {e}")


## 3. Boas Práticas de Segurança

### NUNCA Faça Isso

ERRADO: Hardcoded API key
```
client = OpenAI(api_key="sk-1234567890abcdef...")
```

ERRADO: API key em string no código
```
api_key = "sk-1234567890abcdef..."

```

ERRADO: Commit de .env no Git sem .gitignore (sempre adicione .env ao .gitignore)

### Sempre Faça Isso


In [ ]:
# CORRETO: Usar variáveis de ambiente
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# CORRETO: Verificar se a chave existe
if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("OPENAI_API_KEY não encontrada no ambiente")

print("Boas práticas implementadas!")


---

## 4. Gerenciamento de Custos

### Configurar Limite de Gastos

1. Acesse https://platform.openai.com/account/billing/limits
2. Configure limite mensal ou por requisição
3. Ative alertas de uso

### Monitorar Uso

Acompanhe seu uso em: https://platform.openai.com/usage


In [ ]:
# Exemplo: Monitorar tokens usados
def fazer_requisicao_com_monitoramento(client, prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100
    )
    
    # Monitorar uso
    usage = response.usage
    print(f"Tokens usados:")
    print(f"  - Prompt: {usage.prompt_tokens}")
    print(f"  - Resposta: {usage.completion_tokens}")
    print(f"  - Total: {usage.total_tokens}")
    
    # Calcular custo estimado (GPT-3.5-turbo)
    custo_entrada = (usage.prompt_tokens / 1_000_000) * 0.50
    custo_saida = (usage.completion_tokens / 1_000_000) * 1.50
    custo_total = custo_entrada + custo_saida
    
    print(f"\nCusto estimado: ${custo_total:.6f}")
    
    return response

print("✅ Função de monitoramento criada!")


---

## 5. Tratamento de Erros de Autenticação

### Erros Comuns
